### BUILDING A DATABASE FOR CRIME REPORTS IN BOSTON
In this project we are going to build database for crimes reported in Boston. 
* Our database will be named **crimes_db**
* There will be a schema named **crimes** in **crimes_db** database.
* A table named **boston_crimes** will be created.This table will be loaded with data in boston.csv file.
* Two groups, **readonly** and **readwrite** will be created.There groups will be given appropriate privileges.One user for each group will be created.

The dataset we will use is available in the file boston.csv

### Creating the database and table

First, we will create the database and the schema:

In [1]:
import psycopg2
conn=psycopg2.connect(dbname="dq", user="dq")
cur=conn.cursor()
conn.autocommit=True
cur.execute("DROP DATABASE crime_db;")
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit=False
conn.close()
conn=psycopg2.connect(dbname="crime_db", user="dq")
cur=conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

Now, we will open the file boston.csv and read the first ten row

In [2]:
import csv
counter=0
with open("boston.csv", "r") as file:
    col_headers=next(file)
    reader=csv.reader(file)
    first_row=next(reader)
    print(col_headers)
    #print(first_row)
    for row in enumerate(reader):
        print(row)
        counter += 1
        if counter >= 10:
            break

incident_number,offense_code,description,date,day_of_the_week,lat,long

(0, ['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035'])
(1, ['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943'])
(2, ['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441'])
(3, ['5', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.27536542', '-71.09036101'])
(4, ['6', '3820', 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', '2018-09-03', 'Monday', '42.29019621', '-71.07159012'])
(5, ['7', '724', 'AUTO THEFT', '2018-09-03', 'Monday', '42.30607218', '-71.0827326'])
(6, ['8', '3301', 'VERBAL DISPUTE', '2018-09-03', 'Monday', '42.32701648', '-71.10555088'])
(7, ['9', '301', 'ROBBERY - STREET', '2018-09-03', 'Monday', '42.33152148', '-71.07085307'])
(8, ['10', '3301', 'VERBAL DISPUTE', '2018-09-03', 'Monday', '42.29514664', '-71.05860832'])
(9, ['11', '3301', 'VERBAL DISPUTE', '2018-09-03', 'Monday', '4

We will write an auxiliary function to find distinct values in a given column and csv file:

In [3]:
import csv 
def get_col_set(csv_filename, col_index):
    
    set_for_col=set()
    with open(csv_filename,"r") as file:
        next(file)
        reader=csv.reader(file)
        for row in reader:
            value=row[col_index]
            set_for_col.add(value)
    return set_for_col

for index in range(7):
    set_for_index=get_col_set("boston.csv",index) 
    print("Number of items at column index " 
            + str(index) + ":"
            + str(len(set_for_index)))
    #for item in set_for_index:
          #print(item)
    
            

Number of items at column index 0:298329
Number of items at column index 1:219
Number of items at column index 2:239
Number of items at column index 3:1177
Number of items at column index 4:7
Number of items at column index 5:18177
Number of items at column index 6:18177


Now, we will find the maximum length of each text column in our dataset. There are two textual columns,namely, the _ description and _ day_of_the_week. Since we can tell that the longest of the days is _ Wednesday.We need to compute only the description column.

In [4]:
print(col_headers)

col_values=get_col_set("boston.csv",2)
longest=0
for item in col_values:
    if len(item) > longest:
        longest=len(item)
print("Max length:" + str(longest))

incident_number,offense_code,description,date,day_of_the_week,lat,long

Max length:58


We will create a table inside the crimes schema of the crime_db database.Before creating the table we need to decide the column names and types. We dediced the column names and types as below based on the exploration we made in previous steps:

* incident_number integer PRIMARY KEY
* offense_code integer
* offense_description varchar(60)
* crime_date date
* crime_day enum
* latitute decimal
* longtitute decimal

In [5]:
cur.execute(" CREATE TYPE enum_days AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

In [6]:
cur.execute("CREATE TABLE crimes.boston_crimes (incident_number integer PRIMARY KEY,offense_code integer,offense_description varchar(60),crime_date date,crime_day enum_days, latitute decimal,longtitute decimal)")

### Loading csv to database table

We will load the boston.csv file into table boston_crimes

In [7]:
import csv
with open("boston.csv", "r") as file:
    try:
        cur.copy_expert("COPY crime_db.crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)     
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)

### Creating the necessary groups and users

We will create two groups, _ readonly and _ readwrite but because security is the most important issue in databases, we must be sure that everyone conforms to the least privilege principle.

In [8]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [9]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [10]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT,INSERT,DELETE,UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

We will create two users for the groups we just created and add them to the groups they belong to.

In [11]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

### Testing the database

Now that our database is ready.We need to be sure everything is created properly.First of all, lets make sure that boston_crimes table is created.

In [12]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 0;")
print(cur.description)

(Column(name='incident_number', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='offense_code', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='offense_description', type_code=1043, display_size=None, internal_size=60, precision=None, scale=None, null_ok=None), Column(name='crime_date', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='crime_day', type_code=16418, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='latitute', type_code=1700, display_size=None, internal_size=-1, precision=65535, scale=65535, null_ok=None), Column(name='longtitute', type_code=1700, display_size=None, internal_size=-1, precision=65535, scale=65535, null_ok=None))


Check if boston.csv is loaded to the boston_crimes table. 

In [13]:
cur.execute("SELECT  * FROM crimes.boston_crimes LIMIT 10;")
print(cur.fetchall())

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053')), (2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035')), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943')), (4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441')), (5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101')), (6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.29019621'), Decimal('-71.07159012')), (7, 724, 'AUTO THEFT', datetime.date(2018, 9, 3), 'Monday', Decimal('42.30607218'), Decimal('-71.0827326')), (8, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.32701648'), Decimal('-71.10555088')), (9, 301, 'ROBBERY - STREET', datetime.da

Check if the groups have the necessary privilages:

In [14]:
cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee IN ('readonly','readwrite')")
print(cur.fetchall())

[('readonly', 'SELECT'), ('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]


In [15]:
#commit and close the connection
conn.commit()
conn.close()